In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Reshape, Conv2D, Conv1D, MaxPooling2D, Input, Concatenate
from tensorflow.keras.models import Model
from tensorflow import keras, config
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print("GPUs Available: ", len(config.list_physical_devices('GPU')))

In [ ]:
# Global settings
IMAGE_TARGET_SIZE = (5, 1400, 1)
BATCH_SIZE =  128

In [ ]:

%load_ext autoreload
%autoreload 2

from morse_helpers import create_sets
from morse_label_funcs import  labels_to_one_hot_positions_categorical, letter_n_to_one_hot_positions_categorical, position_regression


def get_sets():
    return [
        # ["./training_data/MorseTrainSet_04/GEN04_VER_000/", 'wordsMatrices_04_000', "Words_04_000.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_100/", 'wordsMatrices_04_100', "Words_04_100.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_200/", 'wordsMatrices_04_200', "Words_04_200.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_210/", 'wordsMatrices_04_210', "Words_04_210.csv"],
        # ["./training_data/MorseTrainSet_04/GEN04_VER_220/", 'wordsMatrices_04_220', "Words_04_220.csv"],

        # ["./training_data/MorseTrainSet_11/GEN11_VER_000/", 'wordsMatrices_11_000', "Words_11_000.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_100/", 'wordsMatrices_11_100', "Words_11_100.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_200/", 'wordsMatrices_11_200', "Words_11_200.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_210/", 'wordsMatrices_11_210', "Words_11_210.csv"],
        # ["./training_data/MorseTrainSet_11/GEN11_VER_220/", 'wordsMatrices_11_220', "Words_11_220.csv"],

        # ["./training_data/MorseTrainSet_06/GEN06_VER_000/", 'wordsMatrices_06_000', "Words_06_000.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_100/", 'wordsMatrices_06_100', "Words_06_100.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_200/", 'wordsMatrices_06_200', "Words_06_200.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_210/", 'wordsMatrices_06_210', "Words_06_210.csv"],
        # ["./training_data/MorseTrainSet_06/GEN06_VER_220/", 'wordsMatrices_06_220', "Words_06_220.csv"],

        # ["./training_data/MorseTrainSet_13/GEN13_VER_000/", 'wordsMatrices_13_000', "Words_13_000.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_100/", 'wordsMatrices_13_100', "Words_13_100.csv"],
        # ["./training_data/MorseTrainSet_13/GEN13_VER_200/", 'wordsMatrices_13_200', "Words_13_200.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_210/", 'wordsMatrices_13_210', "Words_13_210.csv"],
        ["./training_data/MorseTrainSet_13/GEN13_VER_220/", 'wordsMatrices_13_220', "Words_13_220.csv"],

    ]

(image_fnames, morse_labels) = create_sets(
    get_sets(), 
    IMAGE_TARGET_SIZE,
    [position_regression, letter_n_to_one_hot_positions_categorical],
    letter_n=1,
    overwrite_images=False
)

In [ ]:
# Concat label arrays
morse_labels_concat = np.array([morse_labels[0], morse_labels[1].astype("int")]).T

In [ ]:
# Improve me
def labels_to_one_hot(lbls, image_target_size = None):
    assert type(lbls[0]) == np.float64
    label_letters = lbls.astype("int")
    # clean up magic numbers
    labels_one_hot = np.zeros((label_letters.size, 26))
    labels_one_hot[np.arange(label_letters.size),label_letters] = 1
    return labels_one_hot

In [ ]:
from morse_helpers import create_all_sets

TEST_SPLIT_SIZE = 0.80
VALIDATION_SPLIT_SIZE = 0.90

train, labels, train_validation, labels_validation, train_test, labels_test = create_all_sets(
    image_fnames, morse_labels_concat, TEST_SPLIT_SIZE, VALIDATION_SPLIT_SIZE, shuffle_before_test_split=True)


In [ ]:
from morse_helpers import Image_Generator, add_zeropad_random, add_noise_random, return_label_positions

image_prepocessors = [{"func" :add_zeropad_random, "params": [-10, 15]}, {"func": add_noise_random, "params": [0, 20]}]

training_batch_generator = Image_Generator(train, labels, BATCH_SIZE, IMAGE_TARGET_SIZE, image_prepocessors, return_label_positions)
validation_batch_generator = Image_Generator(train_validation, labels_validation, BATCH_SIZE, IMAGE_TARGET_SIZE, image_prepocessors, return_label_positions)


In [ ]:
def get_addblock(x, kernelsize, filters):
    fx = layers.Conv2D(filters, kernelsize, activation='relu', padding='same')(x)
    fx = layers.BatchNormalization()(fx)
    fx = layers.Conv2D(filters, kernelsize, padding='same')(fx)
    out = layers.Add()([x,fx])
    out = layers.ReLU()(out)
    out = layers.BatchNormalization()(out)
    return out

In [ ]:
def conv_model_pos(input_layer):
    cropped = keras.layers.Cropping2D(cropping=((0, 0), (0,1250)), data_format=None)(input_layer)

    conv1           = Conv2D(90,(1,7), padding="same",activation="relu")(cropped)
    pool1           = MaxPooling2D(pool_size=(2,1),padding="same")(conv1)

    conv2           = Conv2D(90,(1,7),padding="same",activation="relu")(pool1)
    pool2           = MaxPooling2D(pool_size=(1,2),padding="same")(conv2)

    conv3           = Conv2D(90,(1,5),padding="same",activation="relu")(pool2)
    pool3           = MaxPooling2D(pool_size=(1,2),padding="same")(conv3)

    conv4           = Conv2D(90,(3,3),padding="same",activation="relu")(pool3)
    pool4           = MaxPooling2D(pool_size=(1,2),padding="same")(conv4)

    conv5           = Conv2D(90,(3,3),padding="same",activation="relu")(pool4)
    pool5           = MaxPooling2D(pool_size=(1,2),padding="same")(conv5)

    conv6           = Conv2D(90,(3,3),padding="same",activation="relu")(pool5)
    pool6           = MaxPooling2D(pool_size=(1,2),padding="same")(conv6)

    conv7           = Conv2D(90,(3,3),padding="same",activation="relu")(pool6)
    pool7           = MaxPooling2D(pool_size=(1,2),padding="same")(conv7)

    flat            = Flatten()(pool7)

    return flat

In [ ]:
from tensorflow.keras import layers

input_layer    = Input(shape=IMAGE_TARGET_SIZE)

flat_position = conv_model_pos(input_layer)

output_layer_position    = Dense(1, name="regr")(flat_position)
model           = Model(inputs=input_layer, outputs=output_layer_position)
model.compile(loss=["mse"], optimizer='adam', metrics=["mean_absolute_error"])

print(model.summary())

init_epoch = 0

In [ ]:

NUM_EPOCHS = 15

def fit_model(epochs):
	
	global init_epoch
	history = model.fit(
					   training_batch_generator,
	                   steps_per_epoch = int(len(train) // BATCH_SIZE),
	                   epochs = epochs + init_epoch,
					   initial_epoch=init_epoch,
	                   verbose =1,
	                   validation_data = validation_batch_generator,
	                   validation_steps = int(len(train_validation) // BATCH_SIZE))
	
	
	init_epoch += epochs
	return history

history = fit_model(NUM_EPOCHS)


In [ ]:
def show_image(img):
    fig = plt.figure(figsize=(30,5))
    plt.xlim(0, 300)
    plt.imshow(img)
    plt.show()
    return fig

In [ ]:
# import sys
# np.set_printoptions(threshold=sys.maxsize)

positions_above_px = 3

def get_deviating_predictions(positions_above_px, generator, predictions):
    regression_differences = []
    indexer = 0

    for imgs_batch, labels_batch in generator:

        for i in range(len(imgs_batch)):

            regr_pred = predictions[indexer] * IMAGE_TARGET_SIZE[1]
            regr_test_label = labels_batch[i] * IMAGE_TARGET_SIZE[1]

            if abs(regr_pred[0] - regr_test_label) > positions_above_px:
                regression_differences.append([regr_pred, regr_test_label, imgs_batch[i]])

            indexer += 1

    return regression_differences

In [ ]:
# TODO : print model summary, model history, image optimizers, training sets used, plot graphs, save to pdf

def print_noise_results(predictions, noise_level, positions_above_px, differences, evaluations):
    print("Total predictions:", len(predictions))
    print("Noise level:", noise_level)
    print("Total position predictions off by more than pixels:", positions_above_px, ":", len(differences))
    print("Position predictions percentage incorrect:", round( (len(differences) / len(predictions) * 100), 4), "%")
    print("Model evaluation:", evaluations)
    print("------------------------------------------------------")

In [ ]:
%%capture cap --no-stderr
from morse_helpers import add_noise, add_noise_random
import inspect

# TODO : create json report instead

print("\nData sets used:")
print(inspect.getsource(get_sets))

print("\nData size:")
print("Training set size:", len(train))
print("Validation set size:", len(train_validation))
print("Test set size:", len(train_test))

print("\nImage pre-processors used:")
print(image_prepocessors)

print("\nConvolution layer:")
print(inspect.getsource(conv_model_pos))

print("\nModel summary:")
print(model.summary())

print("\nModel performance:")
print(history.history)

print("\nTotal epochs:", init_epoch)

#Print results
print("\nResults:")
noise_levels = [0.1, 0.2, 0.3]
for noise_level in noise_levels:

    test_batch_generator = Image_Generator(train_test, labels_test, BATCH_SIZE, IMAGE_TARGET_SIZE, [{"func": add_noise, "params": noise_level}], return_label_positions)
    predictions = model.predict(test_batch_generator)
    regression_differences  = get_deviating_predictions(positions_above_px, test_batch_generator, predictions)
    evaluations = model.evaluate(test_batch_generator, verbose = 0)

    print_noise_results(predictions, noise_level, positions_above_px, regression_differences, evaluations)

In [ ]:
import time
timestr = time.strftime("%Y%m%d-%H%M%S")
with open('results/results.txt' + timestr, 'w') as f:
    f.write(cap.stdout)

In [ ]:

counter = 0   
for diff in regression_differences:

    counter += 1
    if counter > 10:
        break

    pred, correct, img = diff

    img_pred = img.copy()
    img_correct = img.copy()

    print('Prediction', round(pred[0]))
    img_pred[:, round(int(pred))] = 1
    show_image(img_pred)

    print('Correct', round(correct))
    img_correct[:, round(int(correct))] = 1
    show_image(img_correct)

    print("----------------------------------------------------------------------------------------")


In [ ]:
import matplotlib.backends.backend_pdf as mpt

def right_roll_image_by_n(img, roll_value_n):
    return np.roll(img, roll_value_n, axis=1)

def get_label_prediction(img, model, img_width): 
    x_right_rolled_exp_dim = np.expand_dims(img, axis=0)
    single_predict = model.predict(x_right_rolled_exp_dim)
    return single_predict[0] * img_width


def roll_and_display_img(img, indexer):

    image_right_rolled = right_roll_image_by_n(img, int(-10 + (indexer / 5)))

    label_prediction = get_label_prediction(image_right_rolled, model, IMAGE_TARGET_SIZE[1])

    # Draw vertical line
    image_right_rolled[:, int(label_prediction)] = 1

    fig = show_image(image_right_rolled)
    pdf.savefig( fig )


In [ ]:
# # %%capture 
# # capture means suppress output

# pdf = mpt.PdfPages("single_predictions_rolled_randomly.pdf")

# single_image_batch, single_label_batch = test_batch_generator.__getitem__(0)

# for i in range(len(single_image_batch)):
#     roll_and_display_img(single_image_batch[i], i)
    
# pdf.close()